In [15]:
import yfinance as yf
import pandas as pd
import pickle
import numpy as np
import pandas as pd

model = pickle.load(open('adaModel.sav', 'rb'))

In [18]:
df = yf.download(tickers='ADA-USD', period='12h', interval= '15m')
print(len(df))
columns = df.columns.values
# high - low
if not 'Size' in columns:
    df['Size'] = df['High'] - df['Low']
# shift back
if not 'Previous_Close' in columns:
    df['Previous_Close'] = df['Close'].shift(-1)
# size * volume
if not 'VS_multipy' in columns:
    df['VS_multipy'] = df['Size'] * df['Volume']
    df = df[1:-1]
# true range
if not 'True_ranage' in columns:
    df['True_ranage'] = np.abs(df['Close'] - df['Open'])

# cacluate atr 14
if not 'atr_14' in columns:
    high_low = df['High'] - df['Low']
    high_cp = np.abs(df['High'] - df['Close'].shift())
    low_cp = np.abs(df['Low'] - df['Close'].shift())
    df_ = pd.concat([high_low, high_cp, low_cp], axis=1)
    true_range = np.max(df_, axis=1)
    average_true_range = true_range.rolling(14).mean()
    df['atr_14'] = average_true_range.tolist()
    df = df[14:]

# candle colors
if not 'status' in columns:
    conditions = [(df['Close'] > df['Open']), (df['Close'] < df['Open']), (df['Close'] == df['Open'])]
    choices = [1, -1, 0] # up, down, zero
    df['status'] = np.select(condlist=conditions, choicelist=choices, default=0)
# candle types
if not 'candle_type' in columns:
    condistions = [
        (df['True_ranage'] < 0.8 * df['atr_14']), 
        ((df['True_ranage'] > 0.8 * df['atr_14']) & (df['True_ranage'] < 1.2 * df['atr_14'])), 
        ((df['True_ranage'] > 1.2 * df['atr_14']))]
    choices = [0, 1, 2] # 0 --> spining candles, 1 --> standard candles, 2 --> spike candles
    df['candle_type'] = np.select(condlist=condistions, choicelist=choices, default=1)

# atr_55 * previous close
if not 'AP_multipy' in columns:
    df['AP_multipy'] = df['atr_14'] * df['Previous_Close']
# atr_55 * Size
if not 'AS_multipy' in columns:
    df['AS_multipy'] = df['atr_14'] * df['Size']
# atr_55 * Volume
if not 'AV_multipy' in columns:
    df['AV_multipy'] = df['atr_14'] * df['Volume']
if not 'typeSize' in columns:
    df['typeSize'] = df[(df.True_ranage < 0.8 * df.atr_14)]['atr_14'] * 0.8
    df['typeSize'] = df[(df.True_ranage > 0.8 * df.atr_14) & df.True_ranage < 1.2 * df.atr_14]['atr_14']
    df['typeSize'] = df[(df.True_ranage > 1.2 * df.atr_14)]['atr_14'] * 1.2

[*********************100%***********************]  1 of 1 completed
45


/tmp/ipykernel_213466/3246416221.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['True_ranage'] = np.abs(df['Close'] - df['Open'])
/tmp/ipykernel_213466/3246416221.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['atr_14'] = average_true_range.tolist()
/tmp/ipykernel_213466/3246416221.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.py

In [19]:
x = df[['Size', 'Volume', 'VS_multipy', 'AP_multipy', 'AS_multipy']].values
x = x[len(x) - 1]
close = model.predict(x.reshape(1, -1))
print('Close of next Candle : {}'.format(close))

Close of next Candle : [0.18458639]
